# 7. Advanced Deep Learning Best Practices

### Building Dynamic Models Using the Subclassing API

To add flexibility, we can use the Subclassing API to subclass the Model and create the layers needed.

Here, we separate the creating of the layers from their usage.

In [ ]:
class WideAndDeepModel(tf_keras.models.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.hidden_layer1 = tf_keras.layers.Dense(units, activation=activation)
        self.hidden_layer2 = tf_keras.layers.Dense(units, activation=activation)
        self.output_layer = tf_keras.layers.Dense(1)
    
    def call(self, inputs):
        inputa, inputb = inputs
        hidden1 = self.hidden_layer1(inputb)
        hidden2 = self.hidden_layer2(hidden1)
        conct = tf_keras.layers.Concatenate()([inputa, hidden2])
        ouptt = self.output_layer(conct)
        return ouptt
        

In [ ]:
# Load & Train model
model3 = WideAndDeepModel(30, 'relu')
model3.compile(optimizer='sgd', loss='mean_squared_error', metrics=['mae'])
model3.fit((x_train__trainA, x_train__trainB), y_train__train, epochs=20,
           validation_data=((x_train__val_A, x_train__val_B), y_train__val), verbose=0)

In [ ]:
# Evaluate & Predict
model3.evaluate((x_testA, x_testB), y_test)
model3.predict((x_testA[:2], x_testB[:2]))

### Saving & Restoring a Model

This is useful when models take a long time to train or when you need access to a previously trained model.

In [ ]:
# Saving a model
model1.save('model3.h5')

In [ ]:
# Load & Predict
model1ld = tf_keras.models.load_model('model3.h5')
model1ld.predict((x_testA[10:15], x_testB[10:15]))

### Callbacks

Callbacks are useful to perform actions during training. For example, say we want to save the best model during training.

In [ ]:
input_layer = tf_keras.layers.Input(shape=NUM_FEATURES)
hidden_layer1 = tf_keras.layers.Dense(30, activation='relu')(input_layer)
hidden_layer2 = tf_keras.layers.Dense(30, activation='relu')(hidden_layer1)
concat_layer = tf_keras.layers.Concatenate()([input_layer, hidden_layer2])
output_layer = tf_keras.layers.Dense(1)(concat_layer)
model0a = tf_keras.models.Model(inputs=[input_layer], outputs=output_layer)
model0a.compile(optimizer='sgd', loss='mean_squared_error', metrics=['mae'])

# Adding a callback to save only the best model
save_best_checkpoint = tf_keras.callbacks.ModelCheckpoint('model0a_best.h5', save_best_only=True)
model0a.fit(x_train, y_train,  epochs = 10, validation_data=(x_train__val, y_train__val), 
            callbacks=[save_best_checkpoint], verbose=0)

In [ ]:
# Adding a callback to Early Stop to avoid wasting time and resources
# with no further optimisation
stop_early_checkpoint = tf_keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# Combine both callbacks. Use large epoch number because the model will stop when there 
# is no more better performance in the metrics
model0a.fit(x_train, y_train,  epochs = 100, validation_data=(x_train__val, y_train__val), 
            callbacks=[save_best_checkpoint, stop_early_checkpoint], 
            verbose=0)

### Visualisation using TensorBoard

In [ ]:
import os

In [ ]:
def get_run_logdir(root_logdir):
    import time
    run_id = time.strftime("r_%Y%m%d_%H%M%S")
    return os.path.join(root_logdir, run_id)

root_logdirp = os.path.join(os.curdir, "logs")
run_logdir = get_run_logdir(root_logdirp)
print(run_logdir)

In [ ]:
# Create the Tensorboard callback and use it
tensorboard_cb = tf_keras.callbacks.TensorBoard(run_logdir)
model0a.fit(x_train, y_train,  epochs = 100, validation_data=(x_train__val, y_train__val), 
            callbacks=[save_best_checkpoint, tensorboard_cb], 
            verbose=0)

Finally, you can access the TensorBoard with `python -m tensorboard.main --logdir=r_20200601_122625/`

<img src="img3c.png" width="750"/>

Additional Readings:

- (1)  https://ai.googleblog.com/2016/06/wide-deep-learning-better-together-with.html
- (2)  https://github.com/lutzroeder/Netron